In [ ]:
%load_ext autoreload
%autoreload 2
from use_case.tests import * 
from models.eval import *
from models.model import *
from models.trainer import *


In [ ]:
for _ in range(0, 20):
    equilibriua = []
    while len(equilibriua) == 0:
        seed = np.random.randint(0, 1e5)
        env = initialize_baseline(seed = seed)

        payoff_i = env.payoff_i
        payoff_j = env.payoff_j
        equilibriua = find_pure_equilibria(payoff_i, payoff_j)

    for eq in equilibriua:
        x, y = eq 
        a = (y[0] + y[1]) / 2

        print(x, a)

    parameters = ParameterSettings(
        n_agents = env.n_agents,
        d_action = env.n_actions, 
        d_obs = env.obs_size, 
        d_traits = env.d_traits,
        d_het_latent = 4, 
        d_beliefs = env.d_beliefs,
        d_relation = env.d_relation, 
        d_message = 4,
        d_comm_state= env.d_comm_state,
        device = "cuda"
    )
    model = PPOModel(parameters)

    training_parameters = TrainingParameters(
        outer_loops = 200,
        
        actor_learning_rate= 1e-4,
        critic_learning_rate = 1e-4,
        hypernet_learning_rate = 1e-4,

        hypernet_jsd_threshold = 3.0,
        hypernet_samples = 3000,
        hypernet_jsd_weight = 1.0,
        hypernet_entropy_weight = 0.01, 
        hypernet_diversity_weight= 1.0,

        sampled_agents_proportion = 1.0,
        experience_sampling_steps = 10,
        experience_buffer_size = 10,

        entropy_coeff = 0.2,

        epsilon_period = 100,
        epsilon_end = 0.1,
        noise_scale = 1.0,

        entropy_target = 0.5,

        eval_temp = -1.0,
        
        # verbose = False,
        device = parameters.device,
        steps_per_epoch = 16,
        checkpoint_interval= 200
    )

    train_model(model, env, training_parameters)


  2%|▏         | 4/200 [01:36<1:17:44, 23.80s/it]